https://github.com/Teichlab/celltypist

In [11]:
import celltypist
import celltypist as ct
from celltypist import models

In [4]:
#Download all the available models.
models.download_models()
#Update all models by re-downloading the latest versions if you think they may be outdated.
models.download_models(force_update = True)

📂 Storing models in /home/kanferg/.celltypist/data/models
⏩ Skipping [1/50]: Immune_All_Low.pkl (file exists)
⏩ Skipping [2/50]: Immune_All_High.pkl (file exists)
💾 Downloading model [3/50]: Adult_COVID19_PBMC.pkl
⏩ Skipping [4/50]: Adult_CynomolgusMacaque_Hippocampus.pkl (file exists)
⏩ Skipping [5/50]: Adult_Human_PancreaticIslet.pkl (file exists)
⏩ Skipping [6/50]: Adult_Human_Skin.pkl (file exists)
⏩ Skipping [7/50]: Adult_Mouse_Gut.pkl (file exists)
⏩ Skipping [8/50]: Adult_Mouse_OlfactoryBulb.pkl (file exists)
⏩ Skipping [9/50]: Adult_Pig_Hippocampus.pkl (file exists)
⏩ Skipping [10/50]: Adult_RhesusMacaque_Hippocampus.pkl (file exists)
⏩ Skipping [11/50]: Autopsy_COVID19_Lung.pkl (file exists)
⏩ Skipping [12/50]: COVID19_HumanChallenge_Blood.pkl (file exists)
⏩ Skipping [13/50]: COVID19_Immune_Landscape.pkl (file exists)
⏩ Skipping [14/50]: Cells_Adult_Breast.pkl (file exists)
⏩ Skipping [15/50]: Cells_Fetal_Lung.pkl (file exists)
⏩ Skipping [16/50]: Cells_Human_Tonsil.pkl (file

In [6]:
model = models.Model.load()

In [8]:
import cupy as cp
import cupyx
import scanpy as sc
import numpy as np
import pandas as pd
from cupyx.scipy.sparse import csr_matrix
import os
from PIL import Image
from sklearn.linear_model import LinearRegression
import pickle
import pickle
import esda
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import libpysal as lps
from libpysal.weights import W
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import seaborn as sns 
from scipy.sparse import csr_matrix
import pickle
path = "/data/kanferg/Sptial_Omics/playGround/Data/Xenium/output_temp"
pathout = "/data/kanferg/Sptial_Omics/SpatialOmicsToolkit/out_1"
FilePrefix = "_072824" 
grid = sc.read_h5ad(os.path.join(pathout, "grid_save.h5ad"))
file_path = os.path.join(pathout, "grid_uns_mtracies.pkl")

if os.path.getsize(file_path) > 0:
    with open(file_path, 'rb') as buff:
        grid_uns_mtracies = pickle.load(buff)
else:
    print("File is empty. Cannot load data.")
    grid_uns_mtracies = None
grid.uns = {}
grid.uns['cluster'] = pd.read_csv(os.path.join(pathout, "cluster.csv"))
grid.uns['grid_counts'] = grid_uns_mtracies['grid_counts']
grid.uns['grid_xedges'] = grid_uns_mtracies['grid_xedges']
grid.uns['grid_yedges'] = grid_uns_mtracies['grid_yedges']
grid.uns['lrfeatures'] = pd.read_csv(os.path.join(pathout, "lrfeatures.csv"))
grid.uns['lr_summary'] = pd.read_csv(os.path.join(pathout, "lr_summary.csv"))
sparse_matrix = grid.X
row_sums = sparse_matrix.sum(axis=1)
grid.obs['n_counts'] = np.array(row_sums).flatten()

In [13]:
grid.layers['counts'] = grid.X.copy()
sc.pp.normalize_total(grid)
sc.pp.log1p(grid)
grid.layers['log'] = grid.X.copy()

In [17]:
predictions = ct.annotate(grid, model=model, majority_voting=False, over_clustering="cluster")
# convert back to anndata||
adata = predictions.to_adata()

⚠️ Warning: invalid expression matrix, expect ALL genes and log1p normalized expression to 10000 counts per cell. The prediction result may not be accurate
🔬 Input data has 35389 cells and 1500 genes
🔗 Matching reference genes in the model
🧬 775 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!


In [18]:
adata.obs['majority_voting']

grid_112      Endothelial cells
grid_114      Endothelial cells
grid_115      Endothelial cells
grid_116      Endothelial cells
grid_122      Endothelial cells
                    ...        
grid_62417    Endothelial cells
grid_62420          Fibroblasts
grid_62422          Fibroblasts
grid_62423          Fibroblasts
grid_62444          Fibroblasts
Name: majority_voting, Length: 35389, dtype: category
Categories (3, object): ['Endothelial cells', 'Fibroblasts', 'Mast cells']